In [ ]:
!mkdir templates static 
!touch templates/index.html static/script.js static/style.css

In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!ngrok config add-authtoken YOUR_NGROK_KEY
!pip install -U flask-cors

In [ ]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
import os
import io
import base64
import queue
import threading

from flask import Flask, render_template, request, jsonify, send_file
from pyngrok import ngrok
from flask_cors import CORS

request_queue = queue.Queue()

app = Flask(__name__)
cors = CORS(app, resources={r"/*": {"origins": "*"}})
port = 5000



public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

def generate_image(prompt):
  try:
    image = pipe(prompt).images[0]
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue())
    b = "data:image/png;base64," + str(img_str)[2:-1]
    return b
  except Exception as e:
    return str(e)


def process_requests():
    while True:
      try:
        req, response_queue = request_queue.get()
        # Process the request here and generate the response
        prompt = req["prompt"]
        response = generate_image(prompt)
        response_queue.put(response)
      except Exception as e:
        return str(e)


@app.route("/", methods = ['GET', 'POST'])
def index():
  try:
      if request.method == "GET":

        return render_template("index.html")

      else:
        
          req = request.get_json()
          prompt = req["prompt"]

          if ("prompt" not in req) or (prompt == ''):
              return jsonify({"error": "Please specify a prompt parameter"}), 400
          
          response_queue = queue.Queue()
          request_queue.put((req, response_queue))
          response = response_queue.get()

          return jsonify({"image_raw": response})

  except Exception as e:
    return jsonify(error=404, text=str(e)), 404



threading.Thread(target=process_requests).start()
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


# if __name__ == "__main__":
#     app.run(debug=True)